# Práctica 06: Redes bayesianas

Este código utiliza le implementación del archivo Redesbayesianas.ipynb

Pero aquí definimos los nuevos parametos para la modificación realizada al modelo

Recordemos brevemente como es este algoritmo

- Definir el modelo:
    - Identificar las variables 
    - Especificar las relaciones
    - Construir el grafo

- Cuantificar el modelo:
    - Asignar probabilidades
    - Especificar las CPDs

- Realizar inferencia:
    - Utilizar el teorema de Bayes


## Modificación

La forma de trabajo en este nuevo modelo son la flexibilidad de horarios de trabajo, pensado en un entorno de oficina en casa además de poder implemtentar ayuda como inteligencia artificial para las actividades en el puesto. Toda la explicación esta en el pdf. 

In [2]:
# Importar las librerías necesarias
from pgmpy.models import BayesianNetwork # Se usa BayesianModel en lugar de BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Paso 1: Definir la estructura de la red bayesiana
# Aquí se establece cómo se relacionan las variables dentro de la red.
# Las aristas representan influencias directas entre las variables, modelando las dependencias condicionales.
modelo = BayesianNetwork([
    ('Horas de trabajo', 'Balance trabajo-vida'),  # Las horas de trabajo influyen en el balance trabajo-vida
    ('Balance trabajo-vida', 'Satisfacción laboral')  # El balance trabajo-vida influye en la satisfacción laboral
])

# Paso 2: Definir las CPTs (Tablas de Probabilidad Condicional)
# Cada CPT especifica cómo la probabilidad de una variable se ve influenciada por el estado de sus padres en la red.

# CPT para Horas de trabajo (H)
# Define la distribución inicial de cómo se distribuyen las horas de trabajo sin considerar otras variables.
cpt_h = TabularCPD(variable='Horas de trabajo', variable_card=3,
                   values=[[0.08], [0.65], [0.27]],  # Suposiciones de ejemplo sobre la distribución
                   state_names={'Horas de trabajo': ['largas', 'moderadas', 'cortas']})

# CPT para Balance trabajo-vida (B), dependiente de H
# Establece cómo las horas de trabajo afectan el balance trabajo-vida.
cpt_b = TabularCPD(variable='Balance trabajo-vida', variable_card=2,
                   values=[[0.5, 0.85, 0.88], [0.5, 0.15, 0.12]],  # Probabilidades condicionales basadas en H
                   evidence=['Horas de trabajo'], evidence_card=[3],
                   state_names={'Balance trabajo-vida': ['equilibrado', 'no equilibrado'],
                                'Horas de trabajo': ['largas', 'moderadas', 'cortas']})

# CPT para Satisfacción laboral (S), dependiente de B
# Define cómo el balance trabajo-vida afecta la satisfacción laboral.
cpt_s = TabularCPD(variable='Satisfacción laboral', variable_card=3,
                   values=[[0.88, 0.55], [0.1, 0.35], [0.02, 0.1]],  # Probabilidades condicionales basadas en B
                   evidence=['Balance trabajo-vida'], evidence_card=[2],
                   state_names={'Satisfacción laboral': ['satisfecho', 'neutral', 'insatisfecho'],
                                'Balance trabajo-vida': ['equilibrado', 'no equilibrado']})

# Añadir las CPTs al modelo y verificar su validez
modelo.add_cpds(cpt_h, cpt_b, cpt_s)

# check_model() lo usamos para verificar si un modelo bayesiado es valido, para esto necesitamos asegurarnos de que las CPDs(las tablas de probabilidad
# condicional que en nuestro caso son 3 variables) esten correctamente definidas para que la rede bayesiana que se creo tenga coherencia y sentido, para asi poder obtener
# resultados validos.
#
# Al realizar check_model() si el modelo que se definio es valido, entonces esto nos indica que la relaciones entre las 3 variables de la red bayesiana las hemos
# unido correctamente, y entonces regresa true con el mensaje "El modelo es valido", en caso contrario regresamos false y decimos "El modelos es inválido", esto nos dice que
# lo mas seguro es que las relaciones de la red bayesiana que se definieron se cometieron errores dando lugar a una inconsistencia en las tabalas de probabilidad.
if modelo.check_model():
    print("El modelo es válido.")
else:
    print("El modelo es inválido.")

# Paso 3: Realizar la inferencia
# Se utiliza VariableElimination para consultar la red y realizar inferencias sobre variables de interés.

# Crear un objeto de inferencia
inferencia = VariableElimination(modelo)

# Consultar la probabilidad de estar satisfecho con el trabajo bajo condiciones específicas.
# Esto permite entender cómo factores como las horas de trabajo y el balance trabajo-vida afectan la satisfacción laboral.
print("La probabilidad de estar satisfecho con las modificaciones:")
resultado = inferencia.query(variables=['Satisfacción laboral'],
                             evidence={'Horas de trabajo': 'moderadas', 'Balance trabajo-vida': 'equilibrado'})

print(resultado)


El modelo es válido.
La probabilidad de estar satisfecho con las modificaciones:
+------------------------------------+-----------------------------+
| Satisfacción laboral               |   phi(Satisfacción laboral) |
+====================================+=============================+
| Satisfacción laboral(satisfecho)   |                      0.8800 |
+------------------------------------+-----------------------------+
| Satisfacción laboral(neutral)      |                      0.1000 |
+------------------------------------+-----------------------------+
| Satisfacción laboral(insatisfecho) |                      0.0200 |
+------------------------------------+-----------------------------+
